In [9]:
import pprint
import pandas as pd
import numpy as np
import datetime
from bs4 import BeautifulSoup
import requests
import time
import os
import json

# Covert dataframe to JSON files

In [47]:
df=pd.read_csv('Data/PRAW_new.csv')
ner=pd.read_csv('Data/NER.csv')

In [38]:
df=df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])
df

,subreddit,title,score,id,url,comms_num,created,body,search_type
0,Business,Posts regarding politics,81,kurvl4,https://www.reddit.com/r/business/comments/kur...,0,2021-01-11 01:15:49,"Many of you know, we have a strict no-politics...",new
1,Business,USDOT report shows autonomous trucks will only...,271,m8izf6,https://www.traffictechnologytoday.com/news/au...,58,2021-03-19 15:54:04,NaN,new
2,Business,"NFL announces new TV deals, with Amazon gettin...",314,m8ganl,https://www.washingtonpost.com/sports/2021/03/...,67,2021-03-19 13:37:02,NaN,new
3,Business,Ford to hold some F-150s without certain modul...,197,m8j14u,https://www.autonews.com/manufacturing/ford-ho...,39,2021-03-19 15:56:21,NaN,new
4,Business,Lyft says it just had the most riders in a sin...,625,m7zhs6,https://www.cnbc.com/2021/03/18/lyft-says-it-j...,25,2021-03-18 21:09:32,NaN,new
...,...,...,...,...,...,...,...,...,...
13628,SOPA,Make Congress embarrassed of their support for...,3,sev7u,http://torrentfreak.com/hey-congress-i-visited...,0,2012-04-17 22:30:35,NaN,new
13629,SOPA,Stop Cyber Spying Week Launches Protest,12,se55b,http://www.infozine.com/news/stories/op/storie...,0,2012-04-17 14:16:29,NaN,new
13630,SOPA,"Netflix forms PAC, positions itself to support...",441,rzcrn,http://www.politico.com/news/stories/0412/7492...,113,2012-04-08 17:27:52,NaN,new
13631,SOPA,"Reddit, there's a really simple way that we ca...",4,pls8g,https://www.reddit.com/r/SOPA/comments/pls8g/r...,0,2012-02-12 06:20:24,"I've been running this all day, but I think it...",new


In [50]:
ner=ner.drop(columns=['Unnamed: 0'])
ner

,TEXT,START,END,LABEL,Sentence,Article
0,USDOT,0,5,ORG,USDOT report shows autonomous trucks will only...,m8izf6
1,NFL,0,3,ORG,"NFL announces new TV deals, with Amazon gettin...",m8ganl
2,Amazon,33,39,ORG,"NFL announces new TV deals, with Amazon gettin...",m8ganl
3,Thursdays,48,57,DATE,"NFL announces new TV deals, with Amazon gettin...",m8ganl
4,17,64,66,CARDINAL,"NFL announces new TV deals, with Amazon gettin...",m8ganl
...,...,...,...,...,...,...
23729,a week,31,37,DATE,Several groups Monday launched a week of prote...,se36z
23730,Congress,5,13,ORG,Make Congress embarrassed of their support for...,sev7u
23731,CISPA,47,52,ORG,Make Congress embarrassed of their support for...,sev7u
23732,Iran,53,57,GPE,"Reddit, there's a really simple way that we ca...",pls8g


In [54]:
ner[ner['LABEL']=='ORG']

,TEXT,START,END,LABEL,Sentence,Article
0,USDOT,0,5,ORG,USDOT report shows autonomous trucks will only...,m8izf6
1,NFL,0,3,ORG,"NFL announces new TV deals, with Amazon gettin...",m8ganl
2,Amazon,33,39,ORG,"NFL announces new TV deals, with Amazon gettin...",m8ganl
5,Ford,0,4,ORG,Ford to hold some F-150s without certain modul...,m8j14u
20,Wells Fargo Site Crashes,0,24,ORG,Wells Fargo Site Crashes As Customers Look For...,m7nbez
...,...,...,...,...,...,...
23725,CISPA,26,31,ORG,Stop Online Privacy Act: 'CISPA is US cyber-se...,se5nx
23727,CISPA,5,10,ORG,Stop CISPA!!,sfiw8
23730,Congress,5,13,ORG,Make Congress embarrassed of their support for...,sev7u
23731,CISPA,47,52,ORG,Make Congress embarrassed of their support for...,sev7u


In [55]:
ner[ner['LABEL']=='GPE']

,TEXT,START,END,LABEL,Sentence,Article
8,China,0,5,GPE,"China bans Teslas for military members, employ...",m8uzcg
14,Mexico,35,41,GPE,"Six Flags to reopen theme parks in Mexico, Cal...",m7psit
15,California,43,53,GPE,"Six Flags to reopen theme parks in Mexico, Cal...",m7psit
18,US,38,40,GPE,Google to spend $7 billion in 2021 on US offic...,m7pj5y
23,Atlanta Shootings,60,77,GPE,Asian American Business Leaders and Public Fig...,m8341p
...,...,...,...,...,...,...
23713,America,64,71,GPE,Sir Tim Berners-Lee STOPS CISPA IN BRITIAN. L...,sf3hf
23719,America,243,250,GPE,Sir Tim Berners-Lee STOPS CISPA IN BRITIAN. L...,sf3hf
23724,US,9,11,GPE,CISPA is US ‘cyber-security loophole’,se161
23726,US,35,37,GPE,Stop Online Privacy Act: 'CISPA is US cyber-se...,se5nx


In [56]:
ner[ner['LABEL']=='PERSON']

,TEXT,START,END,LABEL,Sentence,Article
9,Teslas,11,17,PERSON,"China bans Teslas for military members, employ...",m8uzcg
19,Neil Rackham,27,39,PERSON,Fascinating interview with Neil Rackham author...,m8lcr0
37,Brexit,55,61,PERSON,UK tech investment hits record $15bn despite C...,m6v5uc
57,Bitcoin,28,35,PERSON,VISA says it’s developing a Bitcoin and crypto...,m792si
77,10.4B Value,64,75,PERSON,eToro to Go Public Via Merger With SPAC; Combi...,m6hjd4
...,...,...,...,...,...,...
23697,Maria Pallante,27,41,PERSON,"Rep. Zoe Lofgren questions Maria Pallante, U.S...",sgqrf
23706,Tim Berners-Lee,0,15,PERSON,Tim Berners-Lee: demand your data from Google ...,sfr27
23711,Tim Berners-Lee STOPS CISPA,4,31,PERSON,Sir Tim Berners-Lee STOPS CISPA IN BRITIAN. L...,sf3hf
23715,Congressperson,156,170,PERSON,Sir Tim Berners-Lee STOPS CISPA IN BRITIAN. L...,sf3hf


In [39]:
#Save to json and output json file
output = df.to_json(orient="records")
output=json.loads(output)
output

In [52]:
#Save to json and output json file
ner_output = ner.to_json(orient="records")
ner_output=json.loads(ner_output)
ner_output

[{'TEXT': 'USDOT',
  'START': 0,
  'END': 5,
  'LABEL': 'ORG',
  'Sentence': 'USDOT report shows autonomous trucks will only lead to minimal layoffs in the sector',
  'Article': 'm8izf6'},
 {'TEXT': 'NFL',
  'START': 0,
  'END': 3,
  'LABEL': 'ORG',
  'Sentence': 'NFL announces new TV deals, with Amazon getting Thursdays and a 17-game season expected',
  'Article': 'm8ganl'},
 {'TEXT': 'Amazon',
  'START': 33,
  'END': 39,
  'LABEL': 'ORG',
  'Sentence': 'NFL announces new TV deals, with Amazon getting Thursdays and a 17-game season expected',
  'Article': 'm8ganl'},
 {'TEXT': 'Thursdays',
  'START': 48,
  'END': 57,
  'LABEL': 'DATE',
  'Sentence': 'NFL announces new TV deals, with Amazon getting Thursdays and a 17-game season expected',
  'Article': 'm8ganl'},
 {'TEXT': '17',
  'START': 64,
  'END': 66,
  'LABEL': 'CARDINAL',
  'Sentence': 'NFL announces new TV deals, with Amazon getting Thursdays and a 17-game season expected',
  'Article': 'm8ganl'},
 {'TEXT': 'Ford',
  'START': 0,

In [42]:
with open('Output/PRAW_All_RedditPost.json', 'w') as json_file:
    json.dump(output, json_file)

In [57]:
with open('Output/NER_All_RedditPost.json', 'w') as json_file:
    json.dump(ner_output, json_file)

# Upload JSON file to Firebase

In [44]:
with open ('Output/PRAW_All_RedditPost.json') as data_file: 
    reddit_post = json.load(data_file)

In [46]:
reddit_post_put = requests.put('https://reddit-database-a3721-default-rtdb.firebaseio.com/reddit_post.json', json=reddit_post)

In [58]:
with open ('Output/NER_All_RedditPost.json') as data_file: 
    reddit_ner = json.load(data_file)

In [59]:
reddit_ner_put = requests.put('https://reddit-database-a3721-default-rtdb.firebaseio.com/reddit_ner.json', json=reddit_ner)